In [ ]:
!pip install transformers datasets peft numpy
!pip install --upgrade datasets
!pip install tabulate

In [ ]:
import torch
from transformers import EarlyStoppingCallback, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, TrainerCallback
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import set_seed
from tabulate import tabulate
from itertools import product

set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

dataset = load_dataset("ag_news")

def tokenize(batch, tokenizer):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="macro")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

class AccuracyThresholdStoppingCallback(TrainerCallback):
    def __init__(self, target_accuracy=0.85):
        self.target_accuracy = target_accuracy
        self.converged_step = None

    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        if metrics and "eval_accuracy" in metrics:
            acc = metrics["eval_accuracy"]
            print(f"Step {state.global_step}: eval_accuracy={acc:.4f}")
            if acc >= self.target_accuracy:
                print(f"Target accuracy {self.target_accuracy} reached at step {state.global_step}. Stopping training.")
                self.converged_step = state.global_step
                control.should_training_stop = True
        return control

def train_eval_lora(model_name, r, lora_alpha, lora_dropout, target_acc=0.85):
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    full_train = dataset["train"].map(lambda x: tokenize(x, tokenizer), batched=True)
    full_train.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

    full_test = dataset["test"].map(lambda x: tokenize(x, tokenizer), batched=True)
    full_test.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

    base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4).to(device)

    peft_config = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        r=r,
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        bias="none",
        target_modules=["query", "value"]
    )
    model = get_peft_model(base_model, peft_config).to(device)
    model.print_trainable_parameters()

    early_stopping_cb = AccuracyThresholdStoppingCallback(target_accuracy=target_acc)

    training_args = TrainingArguments(
        output_dir=f"./{model_name.replace('/', '_')}_lora_r{r}_alpha{lora_alpha}_drop{lora_dropout}",
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        max_steps=3000,
        num_train_epochs=1,
        logging_dir="./logs",
        logging_steps=50,
        save_total_limit=2,
        fp16=torch.cuda.is_available(),
        eval_strategy="steps",
        eval_steps=50,
        load_best_model_at_end=True,
        metric_for_best_model="eval_accuracy",
        greater_is_better=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=full_train,
        eval_dataset=full_test,
        compute_metrics=compute_metrics,
        callbacks=[early_stopping_cb]
    )

    print(f"\nTraining {model_name} with LoRA params: r={r}, alpha={lora_alpha}, dropout={lora_dropout}")
    trainer.train()

    converged_step = early_stopping_cb.converged_step
    metrics = trainer.evaluate()
    print(f"Final Metrics: {metrics}")

    return metrics["eval_accuracy"], converged_step

def eval_base_model(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    full_test = dataset["test"].map(lambda x: tokenize(x, tokenizer), batched=True)
    full_test.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

    base_model_plain = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4).to(device)

    base_trainer = Trainer(
        model=base_model_plain,
        args=TrainingArguments(output_dir=f"./base_eval_{model_name.replace('/', '_')}", per_device_eval_batch_size=32, fp16=torch.cuda.is_available()),
        eval_dataset=full_test,
        compute_metrics=compute_metrics,
    )
    base_metrics = base_trainer.evaluate()
    return base_metrics["eval_accuracy"]

model_names = ["google/bert_uncased_L-2_H-128_A-2", "prajjwal1/bert-tiny"]

results = []

# Evaluate base models
for model_name in model_names:
    print(f"\nEvaluating base (no LoRA) model: {model_name}")
    base_acc = eval_base_model(model_name)
    print(f"Base accuracy for {model_name}: {base_acc:.4f}")
    results.append({
        "model": model_name,
        "r": None,
        "alpha": None,
        "dropout": None,
        "accuracy": base_acc,
        "converged_step": None,
        "note": "base"
    })

# Grid of r and alpha
r_values = [8, 16, 32]
alpha_values = [256, 512]

# Evaluate all combinations
for model_name in model_names:
    for r, alpha in product(r_values, alpha_values):
        acc, step = train_eval_lora(model_name, r=r, lora_alpha=alpha, lora_dropout=0.0, target_acc=0.84)
        results.append({
            "model": model_name,
            "r": r,
            "alpha": alpha,
            "dropout": 0.0,
            "accuracy": acc,
            "converged_step": step,
            "note": "lora"
        })



In [ ]:

# Final results
print("\n=== Final Results ===")
print(tabulate(results, headers="keys", floatfmt=".4f"))